# Phase 1 Minimal Test - PATSTAT REE Analysis for Claude Code

## 🎯 **Objective**
Create a minimal viable Jupyter notebook to validate core PATSTAT functionality on the EPO TIP platform before proceeding with full-scale implementation. This proof-of-concept focuses on essential database connectivity, basic queries, and simple visualization to establish technical feasibility.

## ✅ **Success Criteria**
- ✅ Successful PATSTAT database connection
- ✅ Basic keyword search returning 5-15 results
- ✅ Basic classification search returning 5-15 results
- ✅ Intersection calculation (expecting 0-5 high-quality results)
- ✅ Simple data visualization creation
- ✅ Clean error handling and informative output

## 📋 **Reference Implementation**
This test builds upon our validated best-of-breed PATSTAT script methodologies, using proven table relationships and defensive programming patterns tested against PATSTAT's actual database structure.

---

**🚀 Let's begin the minimal validation test!**

## Section 1: Database Connection Test (Critical)

**Purpose**: Validate PATSTAT connectivity with minimal overhead

**Key Features**:
- Tests both PROD and TEST environments
- Comprehensive error reporting for debugging
- Simple validation query to confirm access
- Critical foundation for all subsequent tests

In [ ]:
# Minimal PATSTAT connection with comprehensive error reporting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def test_patstat_connection_minimal():
    """
    Test PATSTAT connection with minimal overhead
    """
    
    print("🔬 Testing PATSTAT Connection...")
    
    try:
        from epo.tipdata.patstat import PatstatClient
        from epo.tipdata.patstat.database.models import TLS201_APPLN
        
        # Test PROD environment first, then TEST
        for env in ['PROD', 'TEST']:
            try:
                print(f"  Attempting {env} connection...")
                patstat = PatstatClient(env=env)
                db = patstat.orm()
                
                # Simple validation query
                test_result = db.query(TLS201_APPLN.docdb_family_id).limit(1).first()
                
                if test_result:
                    print(f"  ✅ {env} connection successful!")
                    print(f"  📊 Sample family ID: {test_result.docdb_family_id}")
                    return db, env
                    
            except Exception as e:
                print(f"  ❌ {env} failed: {str(e)[:100]}...")
        
        raise Exception("No PATSTAT environment available")
        
    except ImportError as e:
        print(f"❌ PATSTAT import failed: {e}")
        print("💡 Ensure you're running in EPO TIP environment")
        return None, None
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None, None

# Execute connection test
db, environment = test_patstat_connection_minimal()

if db is None:
    print("\n❌ CRITICAL: Cannot proceed without database connection")
    print("🛠️  Check EPO TIP environment and credentials")
else:
    print(f"\n✅ SUCCESS: Connected to PATSTAT {environment}")
    print("🚀 Proceeding with minimal functionality tests...")

## Section 2: Basic Keyword Search (Low Risk)

**Purpose**: Test basic keyword search with minimal complexity

**Strategy**:
- Abstracts only (most reliable text search)
- Recent patents (2020-2024) for performance
- Simple LIKE operators for compatibility
- Very limited scope (10 results max)

**Expected**: 5-15 REE recycling patents from recent years

In [ ]:
def test_keyword_search_minimal():
    """
    Test basic keyword search with minimal complexity
    """
    
    if db is None:
        print("❌ Skipping keyword test - no database connection")
        return pd.DataFrame()
    
    print("\n🔍 Testing Basic Keyword Search...")
    
    try:
        # Simple keyword search - abstracts only, recent patents, very limited scope
        keyword_query = """
        SELECT DISTINCT 
            app.docdb_family_id,
            app.appln_filing_date,
            app.appln_auth,
            SUBSTR(abstr.appln_abstract, 1, 200) as abstract_sample
        FROM TLS201_APPLN app
        JOIN TLS203_APPLN_ABSTR abstr ON app.appln_id = abstr.appln_id  
        WHERE app.appln_filing_date >= '2020-01-01'
          AND app.appln_filing_date <= '2024-12-31'
          AND LOWER(abstr.appln_abstract) LIKE '%rare earth%'
          AND LOWER(abstr.appln_abstract) LIKE '%recycl%'
        LIMIT 10
        """
        
        keyword_results = pd.read_sql(keyword_query, db.bind)
        
        print(f"  ✅ Keyword search successful: {len(keyword_results)} results")
        
        if len(keyword_results) > 0:
            print(f"  📊 Countries found: {keyword_results['appln_auth'].unique()}")
            print(f"  📅 Date range: {keyword_results['appln_filing_date'].min()} to {keyword_results['appln_filing_date'].max()}")
            
            print("  📝 Sample result:")
            sample = keyword_results.iloc[0]
            print(f"     Family: {sample['docdb_family_id']}")
            print(f"     Country: {sample['appln_auth']}")
            print(f"     Abstract: {sample['abstract_sample']}...")
        else:
            print("  ⚠️  No keyword results found - may need broader search terms")
        
        return keyword_results
        
    except Exception as e:
        print(f"  ❌ Keyword search failed: {e}")
        print(f"     Error type: {type(e).__name__}")
        return pd.DataFrame()

# Execute keyword test
keyword_results = test_keyword_search_minimal()
keyword_families = set(keyword_results['docdb_family_id']) if not keyword_results.empty else set()

print(f"\n📊 Keyword search summary: {len(keyword_families)} unique families")

## Section 3: Basic Classification Search (Low Risk)

**Purpose**: Test basic classification search with proven CPC codes

**Strategy**:
- Simple CPC search using most common REE-related codes
- Broad categories (C22B, H01M, C09K) for higher hit rate
- Same date range for consistency
- Limited results for performance

**Expected**: 5-15 patents in metallurgy, battery, or materials classifications

In [ ]:
def test_classification_search_minimal():
    """
    Test basic classification search with proven CPC codes
    """
    
    if db is None:
        print("❌ Skipping classification test - no database connection")
        return pd.DataFrame()
    
    print("\n🏷️  Testing Basic Classification Search...")
    
    try:
        # Simple CPC search - using most common REE codes
        classification_query = """
        SELECT DISTINCT 
            app.docdb_family_id,
            app.appln_filing_date,
            app.appln_auth,
            cpc.cpc_class_symbol
        FROM TLS201_APPLN app
        JOIN TLS224_APPLN_CPC cpc ON app.appln_id = cpc.appln_id
        WHERE app.appln_filing_date >= '2020-01-01'
          AND app.appln_filing_date <= '2024-12-31'
          AND (
            cpc.cpc_class_symbol LIKE 'C22B%' OR
            cpc.cpc_class_symbol LIKE 'H01M%' OR
            cpc.cpc_class_symbol LIKE 'C09K%'
          )
        LIMIT 10
        """
        
        classification_results = pd.read_sql(classification_query, db.bind)
        
        print(f"  ✅ Classification search successful: {len(classification_results)} results")
        
        if len(classification_results) > 0:
            print(f"  📊 Countries found: {classification_results['appln_auth'].unique()}")
            print(f"  🏷️  CPC codes found: {classification_results['cpc_class_symbol'].unique()[:5]}")
            
            print("  📝 Sample result:")
            sample = classification_results.iloc[0]
            print(f"     Family: {sample['docdb_family_id']}")
            print(f"     Country: {sample['appln_auth']}")
            print(f"     CPC: {sample['cpc_class_symbol']}")
        else:
            print("  ⚠️  No classification results found - may need different CPC codes")
        
        return classification_results
        
    except Exception as e:
        print(f"  ❌ Classification search failed: {e}")
        print(f"     Error type: {type(e).__name__}")
        return pd.DataFrame()

# Execute classification test
classification_results = test_classification_search_minimal()
classification_families = set(classification_results['docdb_family_id']) if not classification_results.empty else set()

print(f"\n📊 Classification search summary: {len(classification_families)} unique families")

## Section 4: Intersection Analysis (Core Methodology Test)

**Purpose**: Test the core intersection methodology for quality assurance

**Key Concept**:
- **Intersection**: Patents found in BOTH keyword AND classification searches (highest quality)
- **Union**: All patents found in EITHER search (comprehensive coverage)
- **Precision Rate**: Intersection/Union ratio (quality indicator)

**Expected**: 0-5 intersection results (normal for minimal test), but methodology validation is the key

In [ ]:
def test_intersection_methodology():
    """
    Test the core intersection methodology for quality assurance
    """
    
    print("\n🎯 Testing Intersection Methodology...")
    
    print(f"  📊 Keyword families: {len(keyword_families)}")
    print(f"  📊 Classification families: {len(classification_families)}")
    
    # Calculate intersection (high-quality results)
    intersection_families = keyword_families.intersection(classification_families)
    union_families = keyword_families.union(classification_families)
    
    print(f"  🎯 High-quality intersection: {len(intersection_families)}")
    print(f"  📈 Total unique families: {len(union_families)}")
    
    precision_rate = 0
    if len(union_families) > 0:
        precision_rate = len(intersection_families) / len(union_families) * 100
        print(f"  📊 Search precision rate: {precision_rate:.1f}%")
        
        if len(intersection_families) > 0:
            print("  ✅ METHODOLOGY VALIDATED: Found high-quality intersection results")
            print(f"     High-quality families: {list(intersection_families)[:3]}..." if len(intersection_families) > 3 else f"     High-quality families: {list(intersection_families)}")
        else:
            print("  ⚠️  No intersection found - this is normal for minimal test")
            print("     Methodology is working, just need broader search terms")
    else:
        print("  ⚠️  No families found in either search")
        print("     May need to adjust search parameters or date range")
    
    return {
        'keyword_families': keyword_families,
        'classification_families': classification_families,
        'intersection_families': intersection_families,
        'union_families': union_families,
        'precision_rate': precision_rate
    }

# Execute intersection test
methodology_results = test_intersection_methodology()

print(f"\n📋 Methodology test complete - intersection analysis functional")

## Section 5: Simple Citation Test (Optional)

**Purpose**: Test basic citation functionality if we have any families

**Strategy**:
- Only run if we found some patent families
- Test with just one family for simplicity
- Use family-level citation table (TLS228) for better coverage
- Limited to 5 results for performance

**Note**: Recent patents may have few/no citations - this is normal

In [ ]:
def test_simple_citation_query():
    """
    Test basic citation functionality if we have any families
    """
    
    if db is None or len(methodology_results['union_families']) == 0:
        print("\n⚠️  Skipping citation test - no families available")
        return pd.DataFrame()
    
    print("\n🌍 Testing Simple Citation Query...")
    
    try:
        # Test with just one family for simplicity
        test_families = list(methodology_results['intersection_families'] or 
                           methodology_results['keyword_families'] or 
                           methodology_results['classification_families'])[:1]
        
        if not test_families:
            print("  ⚠️  No families available for citation test")
            return pd.DataFrame()
        
        print(f"  📝 Testing citations for family: {test_families[0]}")
        
        # Simple family-level forward citation query
        citation_query = f"""
        SELECT 
            fam_cit.cited_docdb_family_id as ree_family,
            fam_cit.docdb_family_id as citing_family
        FROM TLS228_DOCDB_FAM_CITN fam_cit
        WHERE fam_cit.cited_docdb_family_id = {test_families[0]}
        LIMIT 5
        """
        
        citation_results = pd.read_sql(citation_query, db.bind)
        
        print(f"  ✅ Citation query successful: {len(citation_results)} citations found")
        
        if len(citation_results) > 0:
            print("  📝 Sample citation:")
            sample = citation_results.iloc[0]
            print(f"     REE family {sample['ree_family']} cited by family {sample['citing_family']}")
        else:
            print("  📝 No citations found for test family (normal for recent patents)")
        
        return citation_results
        
    except Exception as e:
        print(f"  ❌ Citation test failed: {e}")
        print(f"     Error type: {type(e).__name__}")
        return pd.DataFrame()

# Execute citation test
citation_results = test_simple_citation_query()

print(f"\n📊 Citation test complete - {len(citation_results)} citations found")

## Section 6: Basic Visualization Test (Final Validation)

**Purpose**: Create simple visualization to test plotting capabilities

**Features**:
- Simple bar chart of search results comparison
- Country distribution chart (if data available)
- Clean matplotlib formatting
- Professional appearance for presentations

**Success**: Charts display without errors and show data clearly

In [ ]:
def create_minimal_visualization():
    """
    Create simple visualization to test plotting capabilities
    """
    
    print("\n📊 Testing Basic Visualization...")
    
    try:
        # Create simple bar chart of results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Chart 1: Search results comparison
        categories = ['Keywords', 'Classifications', 'Intersection']
        values = [
            len(methodology_results['keyword_families']),
            len(methodology_results['classification_families']),
            len(methodology_results['intersection_families'])
        ]
        
        bars1 = ax1.bar(categories, values, color=['#2E86AB', '#A23B72', '#F18F01'])
        ax1.set_title('REE Patent Search Results\n(Phase 1 Minimal Test)', fontsize=12, pad=20)
        ax1.set_ylabel('Number of Families')
        ax1.grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for i, v in enumerate(values):
            ax1.text(i, v + 0.1, str(v), ha='center', va='bottom', fontweight='bold')
        
        # Chart 2: Country distribution (if we have results)
        if not keyword_results.empty:
            country_counts = keyword_results['appln_auth'].value_counts().head(5)
            bars2 = ax2.bar(country_counts.index, country_counts.values, color='#F18F01', alpha=0.8)
            ax2.set_title('Patents by Country\n(Keyword Search Results)', fontsize=12, pad=20)
            ax2.set_ylabel('Number of Patents')
            ax2.grid(axis='y', alpha=0.3)
            plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
            
            # Add value labels
            for bar in bars2:
                height = bar.get_height()
                ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{int(height)}', ha='center', va='bottom', fontweight='bold')
        else:
            ax2.text(0.5, 0.5, 'No keyword\nresults found', ha='center', va='center', 
                    transform=ax2.transAxes, fontsize=14, color='gray')
            ax2.set_title('Patents by Country\n(No Data Available)', fontsize=12, pad=20)
        
        plt.tight_layout()
        plt.show()
        
        print("  ✅ Visualization created successfully!")
        print("  📊 Charts display search results and country distribution")
        
        return True
        
    except Exception as e:
        print(f"  ❌ Visualization failed: {e}")
        print(f"     Error type: {type(e).__name__}")
        return False

# Execute visualization test
visualization_success = create_minimal_visualization()

## Section 7: Test Results Summary

**Purpose**: Generate comprehensive test summary for go/no-go decision

**Evaluation Criteria**:
- **✅ PASS**: All critical components working
- **⚠️ PARTIAL**: Some issues but core functionality works
- **❌ FAIL**: Critical failures that must be resolved

**Decision Framework**:
- **All Pass**: Proceed with confidence to Phase 2
- **Partial Success**: Proceed cautiously with adjustments
- **Major Failures**: Stop and debug before proceeding

In [ ]:
def generate_test_summary():
    """
    Generate comprehensive test summary for evaluation
    """
    
    print("\n" + "="*60)
    print("📋 PHASE 1 TEST RESULTS SUMMARY")
    print("="*60)
    
    # Test results
    tests = {
        'Database Connection': db is not None,
        'Keyword Search': not keyword_results.empty,
        'Classification Search': not classification_results.empty,
        'Intersection Methodology': len(methodology_results['intersection_families']) >= 0,  # Always passes if methodology runs
        'Visualization': visualization_success
    }
    
    # Print results
    all_passed = True
    critical_passed = True
    
    print("\n🔍 Individual Test Results:")
    for test_name, passed in tests.items():
        status = "✅ PASS" if passed else "❌ FAIL"
        print(f"   {status} {test_name}")
        
        if not passed:
            all_passed = False
            if test_name in ['Database Connection', 'Intersection Methodology']:
                critical_passed = False
    
    print(f"\n📊 Key Metrics:")
    print(f"   • PATSTAT Environment: {environment or 'Not connected'}")
    print(f"   • Keyword families found: {len(methodology_results['keyword_families'])}")
    print(f"   • Classification families found: {len(methodology_results['classification_families'])}")
    print(f"   • High-quality intersection: {len(methodology_results['intersection_families'])}")
    print(f"   • Search precision rate: {methodology_results['precision_rate']:.1f}%")
    print(f"   • Citation results: {len(citation_results)}")
    
    print(f"\n🎯 Overall Assessment:")
    if all_passed:
        print("✅ PHASE 1 FULLY SUCCESSFUL - Ready for Phase 2 development")
        print("💡 All core PATSTAT functionality validated")
        print("🚀 Proceed with HIGH CONFIDENCE to full implementation")
        assessment = "FULL_SUCCESS"
    elif critical_passed:
        print("⚠️  PHASE 1 PARTIALLY SUCCESSFUL - Proceed with caution")
        print("💡 Core functionality works, some components need attention")
        print("🚀 Proceed with MODERATE CONFIDENCE to Phase 2")
        assessment = "PARTIAL_SUCCESS"
    else:
        print("❌ PHASE 1 CRITICAL FAILURES - Must resolve before proceeding")
        print("🔧 Address critical issues before Phase 2 development")
        print("⚠️  DO NOT PROCEED until core functionality is working")
        assessment = "CRITICAL_FAILURE"
    
    print(f"\n📈 Recommended Next Steps:")
    if assessment == "FULL_SUCCESS":
        print("   1. ✅ Expand to full REE keyword set (15+ terms)")
        print("   2. ✅ Add comprehensive IPC + CPC classification codes")
        print("   3. ✅ Implement advanced citation network analysis")
        print("   4. ✅ Add geographic and temporal trend analysis")
        print("   5. ✅ Create professional reporting framework")
        print("   6. ✅ Prepare for German PATLIB demonstrations")
    elif assessment == "PARTIAL_SUCCESS":
        print("   1. 🔧 Debug and fix failed test components")
        print("   2. ⚠️  Proceed cautiously with broader search terms")
        print("   3. 📊 Monitor performance with larger datasets")
        print("   4. ✅ Implement Phase 2 with additional error handling")
    else:
        print("   1. 🚨 Fix database connection issues immediately")
        print("   2. 🔧 Verify EPO TIP environment configuration")
        print("   3. 📞 Contact EPO support if needed")
        print("   4. 🔄 Re-run Phase 1 test until critical components pass")
        print("   5. ❌ DO NOT proceed to Phase 2 until core issues resolved")
    
    print(f"\n⏰ Test completed at: {pd.Timestamp.now()}")
    print(f"🎯 Assessment: {assessment.replace('_', ' ')}")
    
    return assessment, tests

# Generate final summary
final_assessment, test_results = generate_test_summary()

print(f"\n🌟 Phase 1 Minimal Test Complete!")
print(f"📊 Assessment: {final_assessment.replace('_', ' ')}")

---

## 🎯 **Phase 1 Test Complete - Decision Framework**

### **✅ Success Scenario (Ready for Phase 2)**
- **Database Connection**: Working
- **Search Methods**: Both return 5-15 results each
- **Intersection Methodology**: Functions correctly (even if intersection is empty)
- **Visualization**: Displays correctly
- **Confidence Level**: **HIGH** for full implementation

### **⚠️ Partial Success Scenario (Proceed with Caution)**
- **Database**: Connects but some queries fail
- **Results**: Limited but methodology works
- **Action**: Adjust search parameters, proceed cautiously

### **❌ Failure Scenario (Stop and Debug)**
- **Database**: Connection fails
- **Imports**: PATSTAT library errors
- **Action**: Fix environment issues before proceeding

---

## 🚀 **Risk Mitigation Features**

- **✅ Minimal Complexity**: Reduces failure points
- **✅ Comprehensive Error Reporting**: Quick debugging
- **✅ Graceful Degradation**: Allows partial success
- **✅ Clear Success Criteria**: Go/no-go decision framework

**This Phase 1 test provides the confidence foundation needed before presenting to German PATLIB stakeholders or proceeding with full-scale development.**